In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
This is a sample RNN code.
Using this original code, you could already get a 0.63 accuracy.
Please make some changes instead of submitting the original code.
If you have some questions about the code or data,
please contact M.M. Kuang(kuangmeng@hku.hk) or Y.Q. Deng(yqdeng@cs.hku.hk)
"""
import time
import sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

map_fn = tf.map_fn

In [2]:
#new
def str_2_list(data_list):
    ret_list = []
    for i in range(len(data_list)):
        tmp_list = data_list[i].strip().split(" ")
        tmp_ret_list = [int(tmp_list[7][0]),int(tmp_list[6]),int(tmp_list[5]),int(tmp_list[4]),int(tmp_list[3]),int(tmp_list[2]),int(tmp_list[1]),int(tmp_list[0][1])] #changed
        ret_list.append(tmp_ret_list)
    return ret_list

# Prepare Data(Training and Testing)
filename = "data.txt"
a_list = []
b_list = []
c_list = []


In [3]:
with open(filename, "r") as file:
    filein = file.read().splitlines()
    for item in filein:
        tmp_list = item.strip().split(",")
        a_list.append(tmp_list[0])
        b_list.append(tmp_list[1])
        c_list.append(tmp_list[2])
a_list = str_2_list(a_list)
b_list = str_2_list(b_list)
c_list = str_2_list(c_list)

In [4]:
# Define the dataflow graph
time_steps = 8        # time steps which is the same as the length of the bit-string
input_dim = 2         # number of units in the input layer
hidden_dim = 24      # number of units in the hidden layer
output_dim = 1        # number of units in the output layer
binary_dim = 8
largest_number = pow(2, binary_dim)
TINY          = 1e-6    # to avoid NaNs in logs

In [5]:
tf.reset_default_graph()
# input inputs and target ouput Y
inputs = tf.placeholder(tf.float32, [time_steps, None, input_dim], name='x')
outputs = tf.placeholder(tf.float32, [time_steps, None, output_dim], name='y')

# define the RNN cell: can be simple cell, LSTM or GRU
cell = tf.contrib.rnn.BasicLSTMCell(hidden_dim, state_is_tuple=True)
# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim, activation=tf.nn.tanh)

batch_size    = tf.shape(inputs)[1]
initial_state = cell.zero_state(batch_size, tf.float32)

# Given inputs (time, batch, input_size) outputs a tuple
#  - outputs: (time, batch, output_size)  [do not mistake with output_dim]
#  - states:  (time, batch, hidden_size)
rnn_outputs, rnn_states = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, time_major=True)

# project output from rnn output size to output_dim. Sometimes it is worth adding
# an extra layer here.
final_projection = lambda x: layers.linear(x, num_outputs=output_dim, activation_fn=tf.nn.sigmoid)

# apply projection to every timestep.
predicted_outputs = map_fn(final_projection, rnn_outputs)

# # minimize error, using ADAM as weight update rule
error = -(outputs * tf.log(predicted_outputs + TINY) + (1.0 - outputs) * tf.log(1.0 - predicted_outputs + TINY))
error = tf.reduce_sum(error)

learning_rate = 0.06
global_step = tf.Variable(0, trainable=False)
decayed_lr = tf.train.exponential_decay(learning_rate, global_step, 100000, 0.99, staircase=True)
# train_fn = tf.train.AdamOptimizer(decayed_lr).minimize(error, global_step=global_step)
train_fn = tf.train.AdamOptimizer(learning_rate).minimize(error)
accuracy = tf.reduce_mean(tf.cast(tf.abs(outputs - predicted_outputs) < 0.5, tf.float32))

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Number of data for training and testing
#Please remember the total number is 5000
num4train = 3000
num4valid = 1000
num4test = 1000

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# train
batch_size = 500
early_stopping_tolerance = 2
early_stopping_counter = 0
min_valid_error = sys.maxsize


valid_a = np.transpose([a_list[num4train: num4train + num4valid]])
valid_b = np.transpose([b_list[num4train: num4train + num4valid]])
valid_y = np.transpose([c_list[num4train: num4train + num4valid]])
valid_x = np.concatenate((valid_a,valid_b), axis=2)

num_of_epochs = 0
start_time = time.time()
while True:
    num_of_epochs = num_of_epochs + 1
    train_error = 0
    ITERATIONS_PER_EPOCH = int(num4train/batch_size)
    for j in range(ITERATIONS_PER_EPOCH):
        a = np.transpose([a_list[j*batch_size : (j+1)*batch_size]])
        b = np.transpose([b_list[j*batch_size : (j+1)*batch_size]])
        y = np.transpose([c_list[j*batch_size : (j+1)*batch_size]])
        x = np.concatenate((a,b), axis=2)

        train_error += sess.run([error, train_fn], {
            inputs: x,
            outputs: y,
        })[0]
    train_error /= ITERATIONS_PER_EPOCH
    valid_accuracy = sess.run(accuracy, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    valid_error = sess.run(error, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    print("Epoch ", num_of_epochs, ", train err: ", train_error, "valid err: ", valid_error, ", valid acc: ", valid_accuracy * 100.0)
    #early stopping
    if valid_error < 1:
            #early stop
            break
            
#     if valid_error < min_valid_error:
#         min_valid_error = valid_error
#         early_stopping_counter = 0
#     else:
#         early_stopping_counter = early_stopping_counter + 1
#         print("early_stopping_counter: " + str(early_stopping_counter))
#         if early_stopping_counter >= early_stopping_tolerance:
#             #early stop
#             break
end_time = time.time()

remain_result = []
print("Training Finished")
print("Training time: ", end_time - start_time)

Epoch  1 , train err:  2841.594970703125 valid err:  5586.0596 , valid acc:  52.549999952316284
Epoch  2 , train err:  2750.600341796875 valid err:  5434.33 , valid acc:  58.412498235702515
Epoch  3 , train err:  2661.794230143229 valid err:  5309.327 , valid acc:  62.4750018119812
Epoch  4 , train err:  2561.9984130859375 valid err:  5107.8604 , valid acc:  73.67500066757202
Epoch  5 , train err:  2512.9036458333335 valid err:  4909.079 , valid acc:  71.8375027179718
Epoch  6 , train err:  2348.051513671875 valid err:  4284.146 , valid acc:  80.44999837875366
Epoch  7 , train err:  2122.103963216146 valid err:  3510.8643 , valid acc:  80.96250295639038
Epoch  8 , train err:  2015.3983561197917 valid err:  3192.5405 , valid acc:  83.38750004768372
Epoch  9 , train err:  1771.2899373372395 valid err:  2652.5698 , valid acc:  89.20000195503235
Epoch  10 , train err:  1515.3944702148438 valid err:  2079.4038 , valid acc:  91.92500114440918
Epoch  11 , train err:  1217.3442891438801 valid 

Epoch  97 , train err:  1.189773549636205 valid err:  1.1391506 , valid acc:  100.0
Epoch  98 , train err:  1.1648610432942708 valid err:  1.1161292 , valid acc:  100.0
Epoch  99 , train err:  1.1407723724842072 valid err:  1.0937617 , valid acc:  100.0
Epoch  100 , train err:  1.1174860497315724 valid err:  1.072061 , valid acc:  100.0
Epoch  101 , train err:  1.0949372748533885 valid err:  1.0509895 , valid acc:  100.0
Epoch  102 , train err:  1.073112467924754 valid err:  1.030452 , valid acc:  100.0
Epoch  103 , train err:  1.0519432425498962 valid err:  1.0106113 , valid acc:  100.0
Epoch  104 , train err:  1.0314427316188812 valid err:  0.9913281 , valid acc:  100.0
Training Finished
Training time:  12.001443147659302


In [7]:
remain_result = []
average_error = 0
#Test
for i in range(num4train + num4valid + 1, num4train + num4valid + num4test):
    a = np.transpose([[a_list[i]]])
    b = np.transpose([[b_list[i]]])
    y = np.transpose([[c_list[i]]])
    x = np.concatenate((a,b), axis=2)
    
    print(a)
    print(b)
    print(y)
    print(x)

    # get predicted value
    [_probs, _error] = sess.run([predicted_outputs, error], {inputs: x, outputs: y})
    probs = np.array(_probs).reshape([8])
    prediction = np.array([1 if p >= 0.5 else 0 for p in probs]).reshape([8])
    # Save the result
    disired_output = y.reshape([binary_dim])
    remain_result.append([prediction, disired_output])

    # calculate error
    y = y.reshape(binary_dim)
    probs = probs.reshape(binary_dim)
    error_value = np.sum(np.absolute(y - probs))
    
    #print the prediction, the right y and the error.
    print("---------------")
    print(prediction)
    print(disired_output)
    print(error_value)
    print("---------------")
    print()

average_error = average_error/num4test
print(average_error)

[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]]
[[[0 0]]

 [[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[1 0]]

 [[1 1]]

 [[0 0]]]
---------------
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 0 1]
0.0010323583052240792
---------------

[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[0 1]]

 [[1 1]]

 [[1 0]]

 [[0 0]]]
---------------
[0 0 1 1 1 0 0 1]
[0 0 1 1 1 0 0 1]
0.002391060748777818
---------------

[[[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]]
[[[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]]
[[[1 1]]

 [[1 1]]

 [[1 1]]

 [[0 1]]

 [[0 0]]

 [[1 1]]

 [[1 0]]

 [[0 0]]]
---

 [[1]]]
[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[0 0]]

 [[1 1]]

 [[1 0]]

 [[0 0]]

 [[1 0]]

 [[1 0]]]
---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.0012415760502335615
---------------

[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[0 1]]

 [[0 1]]

 [[0 1]]

 [[0 1]]]
---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0004424567154046599
---------------

[[[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[1 1]]

 [[1 1]]

 [[0 0]]

 [[0 1]]

 [[0 0]]

 [[0 1]]

 [[0 1]]

 [[0 1]]]
---------------
[0 1 1 1 0 1 1 1]
[0 1 1 1 0 1 1 1]
0.0011

---------------
[0 1 0 1 1 1 1 0]
[0 1 0 1 1 1 1 0]
0.0008287546256724454
---------------

[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]]
[[[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[0 1]]

 [[0 1]]

 [[1 0]]

 [[0 0]]]
---------------
[0 0 1 0 1 1 1 0]
[0 0 1 0 1 1 1 0]
0.0003915080154683892
---------------

[[[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]]
[[[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]]
[[[1 1]]

 [[1 1]]

 [[0 0]]

 [[0 1]]

 [[0 0]]

 [[0 1]]

 [[1 0]]

 [[0 0]]]
---------------
[0 1 1 1 0 1 1 0]
[0 1 1 1 0 1 1 0]
0.0013761110858467873
---------------

[[[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]]
[[[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]


---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.00044445159710448934
---------------

[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[0 1]]

 [[1 0]]

 [[1 0]]]
---------------
[0 0 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1]
0.0006545841424667742
---------------

[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[0 0]]

 [[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[1 0]]]
---------------
[0 0 0 0 1 1 1 1]
[0 0 0 0 1 1 1 1]
0.00037647297057219475
---------------

[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]]
[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]

---------------
[0 1 0 1 0 1 1 0]
[0 1 0 1 0 1 1 0]
0.0016562637260904012
---------------

[[[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]
[[[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[1 0]]

 [[0 0]]

 [[0 1]]

 [[0 0]]]
---------------
[0 1 1 1 1 0 1 0]
[0 1 1 1 1 0 1 0]
0.0027120994636788964
---------------

[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]
[[[0 0]]

 [[0 0]]

 [[1 1]]

 [[1 1]]

 [[1 1]]

 [[0 1]]

 [[0 0]]

 [[0 0]]]
---------------
[0 0 0 1 1 0 1 0]
[0 0 0 1 1 0 1 0]
0.0005223817406658782
---------------

[[[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]


...

---------------
[0 1 0 1 1 1 0 0]
[0 1 0 1 1 1 0 0]
0.0008258118446065055
---------------

[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[0 1]]

 [[1 0]]

 [[0 0]]

 [[0 0]]]
---------------
[0 0 1 0 1 1 0 0]
[0 0 1 0 1 1 0 0]
0.0004888285086508404
---------------

[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0 0]]

 [[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[1 0]]

 [[0 0]]

 [[0 0]]]
---------------
[0 0 0 1 0 1 0 0]
[0 0 0 1 0 1 0 0]
0.0003238530986209298
---------------

[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]


---------------
[0 1 0 1 1 1 1 1]
[0 1 0 1 1 1 1 1]
0.0008987043634078873
---------------

[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[0 0]]

 [[0 0]]

 [[0 1]]

 [[0 1]]

 [[0 1]]

 [[0 1]]]
---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0004424567154046599
---------------

[[[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[1 1]]

 [[1 1]]

 [[0 0]]

 [[0 1]]

 [[0 0]]

 [[0 1]]

 [[0 1]]

 [[0 1]]]
---------------
[0 1 1 1 0 1 1 1]
[0 1 1 1 0 1 1 1]
0.0011506480623211246
---------------

[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]


---------------
[0 1 0 0 0 0 1 1]
[0 1 0 0 0 0 1 1]
0.00144780646860454
---------------

[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[1 0]]

 [[0 0]]

 [[0 1]]]
---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.00046984317577880574
---------------

[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]]
[[[0 0]]

 [[0 0]]

 [[0 0]]

 [[1 1]]

 [[1 1]]

 [[1 1]]

 [[0 1]]

 [[0 0]]]
---------------
[0 0 0 0 1 1 0 1]
[0 0 0 0 1 1 0 1]
0.00045968757029868357
---------------

[[[1]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[1]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]


[0 0 0 1 0 0 1 1]
0.0005749634492531186
---------------

[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[1 0]]

 [[0 0]]

 [[1 0]]]
---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.00044445159710448934
---------------

[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]]
[[[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[1 1]]

 [[1 1]]

 [[0 0]]

 [[1 0]]

 [[0 1]]

 [[1 0]]

 [[1 0]]]
---------------
[0 0 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1]
0.0006545841424667742
---------------

[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]]
[[[0 0]]

 [[0 0]]

 [[0

---------------
[0 1 1 1 1 0 1 0]
[0 1 1 1 1 0 1 0]
0.0027120994636788964
---------------

[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]
[[[0 0]]

 [[0 0]]

 [[1 1]]

 [[1 1]]

 [[1 1]]

 [[0 1]]

 [[0 0]]

 [[0 0]]]
---------------
[0 0 0 1 1 0 1 0]
[0 0 0 1 1 0 1 0]
0.0005223817406658782
---------------

[[[1]]

 [[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[1]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]
[[[1 1]]

 [[0 0]]

 [[1 1]]

 [[1 0]]

 [[1 1]]

 [[0 1]]

 [[0 0]]

 [[0 0]]]
---------------
[0 1 0 0 1 0 1 0]
[0 1 0 0 1 0 1 0]
0.0016383302681788336
---------------

[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[0]]]
[[[0]]

 [[1]]

 [[1]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]

 [[0]]]
[[[0]]

 [[0]]

 [[1]]

 [[1]]

 [[0]]

 [[0]]

 [[1]]


In [263]:
# Get the total accuracy (Please don't change this part)
accuracy = 0
for i in range(len(remain_result)):
    len_ = len(remain_result[i][0])
    tmp_num = 0
    for j in range(len_):
        if remain_result[i][0][j] == remain_result[i][1][j]:
            tmp_num += 1
    accuracy += tmp_num / len_

accuracy /= len(remain_result)

print("Accuracy: %.4f"%(accuracy))

Accuracy: 1.0000


In [149]:
sess.close()